## Count-Min Sketch for Counting Word Frequencies

Authors: Pablo Mollá Chárlez, Pavlo Poiluha and Junjie Chen

We now aim to approximate counting using a Count-Min sketch - as a standalone program, not in spark
streaming. For this, you will have to keep a structure which host $d \times w$ cells - d hash functions over w
values - that you will set using the $\epsilon$ and $\lambda$ parameters, as outlined in the course slides and/or the original paper, available at:

- http://dimacs.rutgers.edu/~graham/pubs/papers/cmencyc.pdf.


### Generating Multiple Hash Functions

The main component here is generating several hash functions. 

In Python, this is can simply be done by generating a large prime number $p$, and two numbers $a$ and $b$ in the range $[1, p - 1]$:

In [2]:
import random
p = 122354367
a = random.randrange(p)
b = random.randrange(p)

def h(x,a,b,p):
    return (a*hash(x)+b)%p

Generating multiple hash functions is equivalent to generating $d$ pairs of $a, b$ numbers.

If we want to restrict the value space to $0 \cdots w−1$ we can simply use modulo:

In [3]:
def h(x,a,b,p,w):
    return ((a*hash(x)+b)%p)%w

### Procedure Explanation

Let's go through a step-by-step illustration of how the Count-Min Sketch data structure fills up and computes estimates based on the insertion of elements (in this case words). 

For simplicity, I will use smaller parameters and explain each step with specific hash functions, although in practice, the parameters would be chosen based on the desired accuracy and confidence level.

#### A. Setup

For this example, let's assume:
- $\fbox{epsilon (ϵ) = 0.2}$ $\longrightarrow$ This implies a width of $w = ceil(1/0.2) = 5$ (Columns of 2D Array = Matrix).
- $\fbox{delta (δ) = 0.2}$ $\longrightarrow$ This implies a depth of $d = ceil(log(1/0.2)) = 2$ (nº of Hash Functions / Rows of Matrix).

So, our Count-Min Sketch will have 2 rows and 5 columns. 

We'll use a small arbitrary prime $p = 11$ for simplicity, and our hash functions for this example will be:

- First hash function: $\textcolor{green}{\text{h1(x) = (3*hash(x) + 1) \% 11 \% 5}}$
- Second hash function: $\textcolor{blue}{\text{h2(x) = (4*hash(x) + 2) \% 11 \% 5}}$

#### B. Hash Functions

Assuming $\textcolor{red}{\text{hash(x)}}$ simply returns a consistent integer value for the word $x$, we will compute our hash functions accordingly. 
For illustration, let's say:

- hash("apple") = 1
- hash("banana") = 2
- hash("orange") = 3

#### C. Step-by-Step Insertions and Updates

1. **Insert "apple"**:
   - h1("apple") = (3*1 + 1) % 11 % 5 = 4 % 5 = 4
   - h2("apple") = (4*1 + 2) % 11 % 5 = 6 % 5 = 1
   - Update positions $(0,4)$ and $(1,1)$.

2. **Insert "banana"**:
   - h1("banana") = 2
   - h2("banana") = 0
   - Update positions $(0,2)$ and $(1,0)$.

3. **Insert another "apple"**:
   - Use the same hash results as the first "apple":
   - Update positions $(0,4)$ and $(1,1)$ again.

4. **Insert "orange"**:
   - h1("orange") = 0
   - h2("orange") = 4
   - Update positions $(0,0)$ and $(1,4)$.

#### D. Visualization of Sketch After Insertions

Here's how the Count-Min Sketch looks after the above insertions:

- Row 0: [1, 0, 1, 0, 2]  $\textcolor{green}{\text{\# Representing counts affected by "orange", "banana", "apple"}}$
- Row 1: [1, 2, 0, 0, 1]  $\textcolor{green}{\text{\# Representing counts affected by "banana", "apple", "orange"}}$



#### E. Querying the Sketch

When querying, say, the frequency of "apple", the sketch will:
- Calculate $\textcolor{green}{\text{h1("apple")}}$ and $\textcolor{blue}{\text{h2("apple")}}$ to get indices $4$ and $1$.
- Look up these positions in the sketch and find the counts as $[2, 2]$.
- Return the minimum value, which is $2$, as the estimated count of "apple", which in this case, matches the actual count.

#### Conclusion

This example illustrates how each element affects multiple positions in the sketch through different hash functions. It shows how counts can overlap due to collisions, and why the minimum value across the hash functions gives us the best estimate to minimize the impact of these collisions. The Count-Min Sketch efficiently handles insertions and queries, making it suitable for large-scale streaming data where space efficiency and quick updates are necessary.

### Implementation

To implement a Count-Min Sketch in Python, which is a probabilistic data structure used for approximate frequency estimation, we will follow these steps:

1. Initializing the Sketch: We set up a 2D array (matrix) with dimensions $d \times w$. These dimensions are determined by the error parameters $\epsilon$ and $\delta$, where:

    - $\epsilon$: This parameter controls the width (w) of the sketch, which determines the accuracy of the frequency estimates. Smaller values of $\epsilon$ lead to wider sketches and more accurate estimates. 

    $$w = ceil(\frac{1}{\epsilon})$$

    - $\delta$: This parameter controls the depth (d) of the sketch, affecting the confidence of the estimates. Smaller values of $\delta$ result in more hash functions (rows), reducing the probability of overestimation. 

    $$d = ceil(ln(\frac{1}{\delta}))$$

2. Choosing Hash Functions: As described, we will generate $d$ different hash functions. Each function will be determined by two randomly chosen coefficients $a$ and $b$ and a large prime number $p$.

3. Updating the Sketch: For each element in the data stream, we update the sketch using the hash functions to map the element to one of the $w$ columns in each of the $d$ rows.

4. Querying the Sketch: To estimate the frequency of an element, we apply all $d$ hash functions and take the minimum value among the corresponding $w$ columns in the sketch.

5. Comparing with Exact Counts: Finally, we compare the frequency estimations from the Count-Min Sketch with the exact counts.

NB: In this implementation, we initialize the Count-Min Sketch with specified $\epsilon$ and $\delta$ values, we use the given method to generate multiple hash functions and update the counts based on incoming data.

Finally, we compare the results of the Count-Min Sketch to actual frequencies, noting that there might be some overestimation due to the probabilistic nature of the Count-Min Sketch.

In [4]:
import random
import math
from collections import defaultdict

class CountMinSketch:
    def __init__(self, epsilon, delta, prime_number):
        self.w = int(math.ceil(1 / epsilon)) # Initialization of the width 'w' of the table based on epsilon (error).
        self.d = int(math.ceil(math.log(1 / delta))) # Initialization of the depth 'd' of the table based on delta (confidence).
        self.p = prime_number  # A large prime number
        self.initialize_hash_parameters()
        self.initialize_counts_table()

    # Function for the generation of 'd' random pairs of parameters for the hash functions.
    def initialize_hash_parameters(self):
        self.hash_params = []
        for i in range(self.d):
            a = random.randint(1, self.p - 1)
            b = random.randint(1, self.p - 1)
            self.hash_params.append((a, b))

    # Function to create a 2D list for counts, initialzing all to zero.
    def initialize_counts_table(self):
        self.counts = []
        for i in range(self.d):
            row = [0] * self.w
            self.counts.append(row)

    # Hash function to map an item 'x' to an index in the range [0, w-1].
    def hash(self, x, a, b, w):
        return ((a * hash(x) + b) % self.p) % w

    # Augmentation of the count for 'item' in the table for each hash function.
    def add(self, item):
        for i in range(self.d):
            a, b = self.hash_params[i]
            index = self.hash(item, a, b, self.w)
            self.counts[i][index] += 1

    # the frequency estimation of 'item'.
    # It takes the minimum count across all hash tables, which provides an approximation.
    def estimate(self, item):
        min_count = float('inf')
        for i in range(self.d):
            a, b = self.hash_params[i]
            index = self.hash(item, a, b, self.w)
            min_count = min(min_count, self.counts[i][index])
        return min_count

def exact_counts(data):
    counts = defaultdict(int)
    for item in data:
        counts[item] += 1
    return counts

### Attempt 1

In [8]:
# Example usage
word_list = [
    "apple", "banana", "cherry", "date", "elderberry", "fig", "grape",
    "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", 
    "papaya", "quince", "raspberry", "strawberry", "tangerine", 
    "ugli", "voavanga", "watermelon", "xigua", "yamamomo", "zucchini"
]

# Generate a list of 100 words with possible repetitions
random_words = [random.choice(word_list) for _ in range(100)]

epsilon = 0.01  # Error
delta = 0.05   # Confidence
prime_number = 122354367
cms = CountMinSketch(epsilon, delta, prime_number)
for item in random_words:
    cms.add(item)

# Comparison of estimated counts with exact counts
exact = exact_counts(random_words)
for item in set(random_words):
    print(f"Exact count of {item}: {exact[item]}, Estimated count: {cms.estimate(item)}")


Exact count of quince: 4, Estimated count: 4
Exact count of fig: 6, Estimated count: 6
Exact count of zucchini: 4, Estimated count: 4
Exact count of lemon: 2, Estimated count: 2
Exact count of banana: 4, Estimated count: 4
Exact count of mango: 2, Estimated count: 2
Exact count of xigua: 5, Estimated count: 5
Exact count of nectarine: 6, Estimated count: 6
Exact count of kiwi: 7, Estimated count: 7
Exact count of apple: 4, Estimated count: 4
Exact count of date: 5, Estimated count: 9
Exact count of watermelon: 1, Estimated count: 1
Exact count of cherry: 3, Estimated count: 3
Exact count of raspberry: 9, Estimated count: 9
Exact count of honeydew: 4, Estimated count: 4
Exact count of tangerine: 2, Estimated count: 2
Exact count of papaya: 3, Estimated count: 3
Exact count of yamamomo: 3, Estimated count: 3
Exact count of elderberry: 4, Estimated count: 10
Exact count of grape: 6, Estimated count: 6
Exact count of ugli: 8, Estimated count: 8
Exact count of orange: 1, Estimated count: 1


### Attempt 2

In [6]:
epsilon = 0.1  # Error
delta = 0.05   # Confidence
prime_number = 122354367
cms = CountMinSketch(epsilon, delta, prime_number)
for item in random_words:
    cms.add(item)

# Comparison of estimated counts with exact counts
exact = exact_counts(random_words)
for item in set(random_words):
    print(f"Exact count of {item}: {exact[item]}, Estimated count: {cms.estimate(item)}")


Exact count of quince: 6, Estimated count: 9
Exact count of fig: 1, Estimated count: 4
Exact count of zucchini: 4, Estimated count: 7
Exact count of lemon: 4, Estimated count: 12
Exact count of banana: 7, Estimated count: 13
Exact count of mango: 8, Estimated count: 11
Exact count of xigua: 5, Estimated count: 5
Exact count of nectarine: 3, Estimated count: 4
Exact count of apple: 3, Estimated count: 9
Exact count of kiwi: 1, Estimated count: 4
Exact count of date: 3, Estimated count: 7
Exact count of cherry: 5, Estimated count: 11
Exact count of watermelon: 3, Estimated count: 9
Exact count of raspberry: 5, Estimated count: 5
Exact count of voavanga: 3, Estimated count: 12
Exact count of honeydew: 3, Estimated count: 9
Exact count of tangerine: 8, Estimated count: 16
Exact count of papaya: 3, Estimated count: 9
Exact count of yamamomo: 3, Estimated count: 4
Exact count of elderberry: 5, Estimated count: 5
Exact count of grape: 4, Estimated count: 18
Exact count of ugli: 3, Estimated c

From the two example usages, we can see how the parameters epsilon (error) and delta (confidence) impact the accuracy of the frequency estimates. When epsilon is smaller (0.01 in the first example), the CountMinSketch provides very accurate estimates of the counts, with most of the estimated counts matching the exact counts. Based on this we can infer that a smaller epsilon leads to more precise frequency estimates.

However, when epsilon is increased to 0.1 in the second example, the estimates become less accurate, and many are significantly higher than the exact counts. It shows that a larger epsilon results in a wider margin of error in the counts.

In both cases, the delta parameter was kept constant, which means the algorithm maintained the same level of confidence in its accuracy across both tests. Essentially, the difference in epsilon showcases a trade-off between accuracy and space efficiency (since a smaller epsilon requires more space).

Overall, these examples demonstrate that while the CountMinSketch can be a highly effective tool for estimating frequencies in large datasets, the choice of parameters critically influences its accuracy. If space allows, choosing a smaller epsilon will provide more accurate results, but this comes at the cost of using more memory.